In [13]:
# Script to calculate the holdings of a hypothetical market-cap weighted crypto-ETF. 

TOP_X_CRYPTOS = 50 # Thresholded up to 100
DONT_INCLUDE_TOP_X = 0
TOTAL_AMOUNT_TO_INVEST = 3000 # dollars

BLACK_LISTED_SYMBOLS = {"USDT", "USDC", "UST", "BUSD"}

In [14]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import requests
from bs4 import BeautifulSoup
import pandas as pd 
import time 
driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST chromedriver version for 96.0.4664 google-chrome
Driver [/Users/daniel/.wdm/drivers/chromedriver/mac64/96.0.4664.45/chromedriver] found in cache
/var/folders/3d/vggd2dl96n937ldjr_8vgn240000gn/T/ipykernel_22909/796157277.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [15]:
def get_soup(url): 
    # https://coderedirect.com/questions/215584/python-3-using-requests-does-not-get-the-full-content-of-a-web-page
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    driver.quit()
    return soup 

def get_soup2(url): 
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    return soup 

In [16]:
url_overrides = {
    "dai" : "multi-collateral-dai",
    "xdc-network" : "xinfin",
    '1inch-network' : '1inch',
    "gno" : "gnosis-gno",
    "pax-dollar": "paxos-standard"
}

In [17]:

url = "https://coinmarketcap.com/"
soup = get_soup(url)
table = soup.find('table', attrs={'class':'h7vnx2-2 czTsgW cmc-table'})
table_body = table.find('tbody')
url =  'https://coinmarketcap.com/'

results = []

rows = table_body.find_all('tr')
for row in rows[DONT_INCLUDE_TOP_X:min(100, TOP_X_CRYPTOS)]:
    # coin_item_symbol = row.find('p', class_ = "sc-1eb5slv-0 gGIpIK coin-item-symbol")
    coin_symbol = row.find('p', attrs={'class':"sc-1eb5slv-0 gGIpIK coin-item-symbol"}) 
    try: 
        if coin_symbol:
            coin_market_cap =row.find('span', attrs={'class': "sc-1ow4cwt-1 ieFnWP"})

        else: 
            coin_symbol = row.find('span', attrs={'class':"crypto-symbol"})
            url_symbol_text = row.find_all('span')[3].text.lower().replace(" ", "-").replace(".", "-")

            if url_symbol_text in url_overrides: 
                url_symbol_text = url_overrides[url_symbol_text]

            url = f'https://coinmarketcap.com/currencies/{url_symbol_text}/'
            market_cap_soup = get_soup2(url) 
            coin_market_cap = market_cap_soup.find_all('div', attrs={"class": "statsItemRight"})[0].find("div", attrs={"class": "statsValue"})

        coin_market_cap_int = int(coin_market_cap.text.replace(",", "").replace("$", ""))
    except: 
        print(f"Failed for token {coin_symbol}")

    if coin_symbol.text in BLACK_LISTED_SYMBOLS: 
        continue 
    results.append({'symbol': coin_symbol.text, 'market_cap': coin_market_cap_int})

In [18]:
investing_df = pd.DataFrame(results)

total_market_cap = investing_df['market_cap'].sum() 
investing_df['weight'] = investing_df['market_cap'] / total_market_cap

In [19]:
investing_df.head()

,symbol,market_cap,weight
0,BTC,875392022663,0.452242
1,ETH,453343419483,0.234205
2,BNB,85046177329,0.043936
3,SOL,52178365678,0.026956
4,ADA,44415527212,0.022946


In [20]:
# https://docs.gspread.org/en/latest/oauth2.html#enable-api-access
# https://docs.gspread.org/en/latest/user-guide.html#opening-a-spreadsheet
from google.oauth2.service_account import Credentials
import gspread

scopes = [
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
]

credentials = Credentials.from_service_account_file(
    'jsonFileFromGoogle.json',
    scopes=scopes
)

gc = gspread.authorize(credentials)

In [21]:
sheet_key = '1J7roesFAznFzOY849R35rzc-n97gRfEL5cezDIzZA9c'
sheet = gc.open_by_key(key=sheet_key)


# https://docs.gspread.org/en/latest/user-guide.html#creating-a-worksheet
worksheet_title = "Crypto ETF (From Script)"
try: 
    worksheet = sheet.worksheet(worksheet_title)
except gspread.WorksheetNotFound:
    worksheet = sheet.add_worksheet(title=worksheet_title, rows=100, cols=20)

# https://docs.gspread.org/en/latest/user-guide.html#using-gspread-with-pandas
worksheet.update([])
worksheet.update([investing_df.columns.values.tolist()] + investing_df.values.tolist())
worksheet.update_cell(1, 5, "Time Last Updated:")
worksheet.update_cell(2, 5, time.ctime())

{'spreadsheetId': '1J7roesFAznFzOY849R35rzc-n97gRfEL5cezDIzZA9c',
 'updatedRange': "'Crypto ETF (From Script)'!E2",
 'updatedRows': 1,
 'updatedColumns': 1,
 'updatedCells': 1}